## DATA IMPORTING

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [60]:
df1 = pd.read_csv('train.csv') #Training Dataset
df2 = pd.read_csv('test.csv') #Testing dataset
codf = pd.read_csv('US_COVID_data_state_level.csv') #Covid data of cities
cidf = pd.read_csv('city_details.csv') #Cities data
wadf = pd.read_csv('warehouse_mapping.csv') #Warehouse data

## Predicting Confirmed Cases

In [61]:
corl = codf[codf.conf_cases.notna()] #Non-empty cells of confirmed cases

In [62]:
a,b = list(corl.tot_cases),list(corl.conf_cases)

In [63]:
from sklearn.ensemble import RandomForestRegressor 
algo=RandomForestRegressor()
ratio = []
for i in range(len(a)):
    if b[i]!=0:
        ratio.append((a[i]-b[i])/(b[i]))
    else:
        ratio.append(0)
a=np.array(a).reshape(-1,1)
algo.fit(a,ratio)

RandomForestRegressor()

In [64]:
for i in codf[codf.conf_cases.isna()].index:
    codf.iloc[i,3] = int(np.array(codf.iloc[i,2])/(1+algo.predict(np.array(codf.iloc[i,2].reshape(1,-1)))))

In [65]:
codf.sample(5)

,date,state,tot_cases,conf_cases
13785,07/09/20,WV,11575,11368.0
17982,16/11/20,IL,585248,585248.0
18603,27/11/20,PA,343614,319239.0
1953,23/02/20,NE,0,0.0
21724,18/01/21,WV,109809,88434.0


## DATA CLEANING

In [108]:
cidf1 = cidf.drop(['country'],axis = 1) #dropping useless columns
cidf1 = cidf1.drop_duplicates()
cidf1.tail(),cidf1.shape

(            city  zip_code state  state_name
 554  WESTMINSTER     92683    CA  CALIFORNIA
 555      ONTARIO     91761    CA  CALIFORNIA
 556       TACOMA     98409    WA  WASHINGTON
 557      BOZEMAN     59718    MT     MONTANA
 558    CLACKAMAS     97015    OR      OREGON,
 (555, 4))

In [109]:
state = cidf1.state.unique()
num_cities = [cidf1[cidf1.state == st].zip_code.nunique() for st in state]
stct = pd.DataFrame({'state':state,'num_cities':num_cities})

In [110]:
value,counts = np.unique(cidf1.zip_code,return_counts=True)
value[counts>1],counts[counts>1]

(array([94080, 98027], dtype=int64), array([2, 2], dtype=int64))

In [111]:
cidf1[cidf1.zip_code == 94080]

,city,zip_code,state,state_name
370,S,94080,CA,CALIFORNIA
371,EL,94080,CA,CALIFORNIA


In [112]:
cidf1.drop(index=371,inplace=True)

In [113]:
cidf1[cidf1.zip_code == 98027]

,city,zip_code,state,state_name
69,ISSAQUAH,98027,WA,WASHINGTON
70,US,98027,WA,WASHINGTON


In [114]:
cidf1.drop(index=70,inplace=True)

## DATA PREPROCESSING

In [115]:
cidf_final = cidf1.merge(stct,on = 'state',how = 'left')

In [116]:
wadf.sample(5)

,wh_id,warehouse_name,city
198,1330,WAREHOUSE 1330 WEBSTER,WEBSTER
195,1325,WAREHOUSE 1325 DAVENPORT,DAVENPORT
184,1298,WAREHOUSE 1298 N SPOKANE,N
541,784,WAREHOUSE 784 GRAND RAPIDS,GRAND
36,1033,WAREHOUSE 1033 POCATELLO,POCATELLO


In [117]:
m = codf.merge(cidf_final,how = 'left',on='state') 
print(m.shape)
m['avg_tot_cases'] = m['tot_cases']/m['num_cities']
m['avg_conf_cases'] = m['conf_cases']/m['num_cities']
m.sample(5) #merging covid data and cities data using state as a common entity

(276208, 8)


,date,state,tot_cases,conf_cases,city,zip_code,state_name,num_cities,avg_tot_cases,avg_conf_cases
168369,14/11/20,MI,389560,356775.0,MIDLAND,48642.0,MICHIGAN,16.0,24347.500000,22298.437500
98341,13/07/20,NJ,177044,172498.0,CLIFTON,7014.0,NEW JERSEY,20.0,8852.200000,8624.900000
171770,20/11/20,ID,89764,76570.0,TWIN,83301.0,IDAHO,7.0,12823.428571,10938.571429
48122,16/04/20,PA,27735,27545.0,SANATOGA,19464.0,PENNSYLVANIA,11.0,2521.363636,2504.090909
241759,24/03/21,AZ,838474,779737.0,N,85027.0,ARIZONA,18.0,46581.888889,43318.722222


In [118]:
b = []
for row in wadf.values:
  b.append(row[1].split()[2]) #taking 3rd word from warehouse_name which is city

wadf['city'] = np.array(b)
wadf.sample(5)

,wh_id,warehouse_name,city
496,689,WAREHOUSE 689 NW SAN ANTONIO,NW
314,343,WAREHOUSE 343 AVON OH,AVON
124,1191,WAREHOUSE 1191 KALAMAZOO,KALAMAZOO
407,48,WAREHOUSE 48 VAN NUYS,VAN
422,671,WAREHOUSE 671 HAWTHORNE,HAWTHORNE


In [119]:
x = df1.merge(wadf.drop(axis = 1,columns=['warehouse_name']),how='left',on = 'wh_id')

In [120]:
print(x.shape)
x.sample(5)
# merged data of train and warehouse datasets

(453215, 4)


,date,wh_id,sales,city
27001,13/03/20,1030.0,389,W
60814,16/03/20,111.0,743,TIGARD
247896,01/02/21,336.0,237,CLEARWATER
258636,21/08/19,351.0,24,FORT
280862,24/12/20,379.0,207,SPRINGDALE


In [121]:
z = x.merge(m,how='left', on = ['date','city'])

In [122]:
print(z.shape)
z.sample(5)

(669515, 12)


,date,wh_id,sales,city,state,tot_cases,conf_cases,zip_code,state_name,num_cities,avg_tot_cases,avg_conf_cases
273515,12/05/19,146.0,17,LIVERMORE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
654532,07/06/20,827.0,392,PHOENIX,AZ,26889.0,26603.0,85017.0,ARIZONA,18.0,1493.833333,1477.944444
233874,26/02/20,1332.0,336,N,FL,0.0,0.0,33181.0,FLORIDA,28.0,0.000000,0.000000
238955,28/12/19,1343.0,0,MERIDIAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260660,21/01/21,1384.0,28,CHAMPAIGN,IL,1086333.0,1086333.0,61820.0,ILLINOIS,22.0,49378.772727,49378.772727


In [123]:
#precovid data
precov = z[z.state.isna()][['date','wh_id','sales','tot_cases','conf_cases','avg_tot_cases','avg_conf_cases']].fillna(0)
cov = z[z.state.notna()][['date','wh_id','sales','tot_cases','conf_cases','avg_tot_cases','avg_conf_cases']]

## EDA

In [ ]:
hii